# Shallow Neural Network [Keras]
---
- Author: Diego Inácio
- GitHub: [github.com/diegoinacio](https://github.com/diegoinacio)
- Notebook: [neural_network_shallow_Keras.ipynb](https://github.com/diegoinacio/machine-learning-notebooks/blob/master/Deep-Learning-Models/neural_network_shallow_Keras.ipynb)
---
Implementation of *Shallow Neural Network* using Keras library.

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

In [ ]:
plt.rcParams['figure.figsize'] = (16, 8)

In [ ]:
cifar10 = tf.keras.datasets.cifar10
[X_train, Y_train],[X_test, Y_test] = cifar10.load_data()

# data cleaning
# using only zeros and ones
X_train = X_train[Y_train.ravel() <= 1]
Y_train = Y_train[Y_train <= 1]
X_test = X_test[Y_test.ravel() <= 1]
Y_test = Y_test[Y_test <= 1]

print('X_train:', X_train.shape)
print('Y_train:', Y_train.shape)
print('X_test:', X_test.shape)
print('Y_test:', Y_test.shape)

In [ ]:
# Label categories
objects = ['Plane', 'Car']
# Get dimensions
N1, N2, C = X_train[0].shape

In [ ]:
fig, AX = plt.subplots(3, 6, sharex=True, sharey=True)

np.random.seed(1234)
for ax in AX.ravel():
    rindex = np.random.randint(Y_train.size)
    ax.imshow(X_train[rindex])
    label = Y_train[rindex]
    ax.set_title(f'label: {objects[label]} ({label})')
plt.grid(False)

In [ ]:
# data preparation
# scales, dimensions and dtypes
x_train, y_train = X_train/255, Y_train[np.newaxis].T
x_test, y_test = X_test/255, Y_test[np.newaxis].T

x_train = x_train.astype(np.float32).reshape(-1, N1*N2*C)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32).reshape(-1, N1*N2*C)
y_test = y_test.astype(np.float32)

print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

## Shallow Neural Network
---
![perceptron](sourceimages/neural_network_shallow.png "Shallow Neural Network")

In [ ]:
EPOCHS = 500  # epochs
BATCH = 1000  # batch size

# Number of neurons on the hidden layer
neurons = 32

# m is the number of examples
# n_x is the input size 28x28=784
m, n_x = x_train.shape

# model
model = tf.keras.Sequential([
    tf.keras.layers.Input(n_x),
    tf.keras.layers.Dense(neurons, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['accuracy']
)

# loss and accuracy storage
loss_plot = []; accA_plot = []

for epoch in range(EPOCHS + 1):
    # randomic batch definition
    rbatch = np.random.choice(Y_train.size, size=BATCH)
    # training, metrics and storage
    model.fit(x_train[rbatch], y_train[rbatch], epochs=1, verbose=0)
    loss_plot += [e*100 for e in model.history.history['loss']]
    accA_plot += [e*100 for e in model.history.history['accuracy']]
    if (not epoch % 100) and (epoch != 0):
        print(f'epoch: {epoch:04d} | loss: {loss_plot[-1]:.3f} | accuracy: {accA_plot[-1]:06.2f} %')

# store W and B for visualization and test
w_ = model.weights[0].numpy()

In [ ]:
fig, [axA, axB] = plt.subplots(2, 1, sharex=True)

axA.plot(loss_plot)
axA.set_ylabel('loss')
axB.plot(accA_plot)
axB.set_ylabel('accuracy')

plt.xlabel('epochs')

plt.show()

In [ ]:
fig, AX = plt.subplots(3, 6, figsize=(2048//72, 1024//72))
AX = [b for a in AX for b in a]

pred = model.predict(x_test)

np.random.seed(1)
for ax in AX:
    index = np.random.randint(y_test.size)
    a_ = pred[index]
    y_ = 1 if a_ > 0.5 else 0
    imshow = x_test[index].reshape(N1, N2, C)
    if y_ == y_test[index]:
        imshow[-4:, -4:] = (0, 1, 0)
        ax.imshow(imshow)
    else:
        imshow[-4:, -4:] = (1, 0, 0)
        ax.imshow(imshow)
    ax.set_title(f'{objects[y_]} ({y_})' + r' ; $a_i$ = {:.02f}'.format(float(a_)), fontsize=20)